In [1]:
import socket
import cv2
import numpy as np
import pickle
import time

In [2]:
qtPort=8187
videoIP="127.0.0.1"
videoPort=8188

In [3]:
# 创建 TCP 客户端套接字（连接到 Qt 程序）
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(("localhost", qtPort))  # 本地端口连接Qt程序

# 创建视频接收的 TCP 服务器套接字（接收远程视频流）
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# 绑定之前设置 SO_REUSEADDR
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

In [4]:

# 尝试连接并接收视频流
while True:
    try:
        # 尝试绑定并监听连接
        server_socket.bind((videoIP, videoPort))
        server_socket.listen(1)
        print("Waiting for video connection...")

        # 等待远程视频流连接
        video_socket, addr = server_socket.accept()
        print(f"Connection established with {addr}")
        
        # 接收视频流并进行处理
        while True:
            data = b""
            while len(data) < 88888:
                packet = video_socket.recv(88888)
                if not packet:
                    break
                data += packet

            if not data:  # 如果没有接收到数据，表示连接已经断开
                print("Connection lost, retrying...")
                break  # 跳出当前接收循环，重新尝试连接

            # 解码图像
            img = np.frombuffer(data, dtype=np.uint8)
            frame = cv2.imdecode(img, cv2.IMREAD_COLOR)

            # 在这里处理图像，例如深度学习识别或其他操作
            # 进行深度学习处理，假设使用 OpenCV 或深度学习框架进行处理
            '''
            # 假设识别结果是一个框，识别出物体并标记
            boxes = [(50, 50, 100, 100, "Object1"), (150, 150, 120, 120, "Object2")]
            for box in boxes:
                x, y, w, h, label = box
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # 绘制框
                cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            '''

            # 编码图像并通过 Qt 发送
            _, img_encoded = cv2.imencode('.jpg', frame)
            img_bytes = img_encoded.tobytes()
            try:
                client_socket.sendall(img_bytes)
            except Exception as e:
                print(f"Error sending image to Qt: {e}")
                break  # 如果发送失败，退出当前循环并重新尝试连接

    except (socket.error, Exception) as e:
        print(f"Error occurred: {e}")
        print("Reattempting to bind and accept connection...")
        server_socket.close()  # 关闭现有连接，准备重新绑定
        time.sleep(3)  # 延时3秒再进行重试
        server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # 重新创建socket
        server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

Waiting for video connection...
Connection established with ('127.0.0.1', 58764)
Error sending image to Qt: [Errno 32] Broken pipe
Error occurred: [Errno 22] Invalid argument
Reattempting to bind and accept connection...


In [1]:
import atexit
import socket
import sys
import time

import cv2
import numpy as np
import torch

In [2]:
# 接受图片大小的信息
def recv_size(sock, count):
    buf = ''
    while count:
        newbuf = sock.recv(count)
        if not newbuf: return None
        buf += newbuf
        count -= len(newbuf)
    return buf


In [3]:
def main():
    print(f"Using torch version: {torch.__version__}")
    sys.stdout.flush()

    qtPort = 8187
    videoIP = "0.0.0.0"
    videoPort = 8388

    # 创建 TCP 客户端套接字（连接到 Qt 程序）
    #client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    #client_socket.connect(("localhost", qtPort))  # 本地端口连接Qt程序

    # 创建视频接收的 TCP 服务器套接字（接收远程视频流）
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    # 绑定之前设置 SO_REUSEADDR
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

    # 在程序结束时，自动调用关闭 socket 的方法
    atexit.register(lambda: server_socket.close())

    # 尝试连接并接收视频流
    while True:
        try:
            # 尝试绑定并监听连接
            server_socket.bind((videoIP, videoPort))
            server_socket.listen(1)
            print("Waiting for video connection...")
            sys.stdout.flush()

            # 等待远程视频流连接
            video_socket, addr = server_socket.accept()
            print(f"Connection established with {addr}")
            sys.stdout.flush()

            # 接收视频流并进行处理
            while True:
                length = recv_size(server_socket, 16)  # 首先接收来自客户端发送的大小信息
                if isinstance(length, str):  # 若成功接收到大小信息，进一步再接收整张图片
                    stringData = recv_size(server_socket, int(length))
                    data = np.fromstring(stringData, dtype='uint8')
                    decimg = cv2.imdecode(data, 1)  # 解码处理，返回mat图片
                    cv2.imshow('SERVER', decimg)
                    if cv2.waitKey(10) == 27:
                        break
                    print('Image recieved successfully!')
                    sys.stdout.flush()
                if cv2.waitKey(10) == 27:
                    break

                cv2.imshow('dada',decimg)

                # 在这里处理图像，例如深度学习识别或其他操作
                # 进行深度学习处理，假设使用 OpenCV 或深度学习框架进行处理
                '''
                # 假设识别结果是一个框，识别出物体并标记
                boxes = [(50, 50, 100, 100, "Object1"), (150, 150, 120, 120, "Object2")]
                for box in boxes:
                    x, y, w, h, label = box
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # 绘制框
                    cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                '''

                '''
                # 编码图像并通过 Qt 发送
                _, img_encoded = cv2.imencode('.jpg', frame)
                img_bytes = img_encoded.tobytes()
                try:
                    client_socket.sendall(img_bytes)
                except Exception as e:
                    print(f"Error sending image to Qt: {e}")
                    sys.stdout.flush()
                    break  # 如果发送失败，退出当前循环并重新尝试连接
                '''

        except (socket.error, Exception) as e:
            print(f"Error occurred: {e}")
            print("Reattempting to bind and accept connection...")
            sys.stdout.flush()
            server_socket.close()  # 关闭现有连接，准备重新绑定
            time.sleep(3)  # 延时3秒再进行重试
            server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # 重新创建socket
            server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)


In [4]:
main()

Using torch version: 1.13.1+cpu
Waiting for video connection...
Connection established with ('192.168.1.11', 36840)
Error occurred: [Errno 107] Transport endpoint is not connected
Reattempting to bind and accept connection...
Waiting for video connection...
Connection established with ('192.168.1.11', 37068)
Error occurred: [Errno 107] Transport endpoint is not connected
Reattempting to bind and accept connection...
Waiting for video connection...
Connection established with ('192.168.1.11', 37348)
Error occurred: [Errno 107] Transport endpoint is not connected
Reattempting to bind and accept connection...
Waiting for video connection...
Connection established with ('192.168.1.11', 37534)
Error occurred: [Errno 107] Transport endpoint is not connected
Reattempting to bind and accept connection...
Waiting for video connection...
Connection established with ('192.168.1.11', 37806)
Error occurred: [Errno 107] Transport endpoint is not connected
Reattempting to bind and accept connection..

KeyboardInterrupt: 

In [5]:
import socket
import numpy as np
import cv2
import pickle
import struct
import time

In [6]:
def start_client(host='127.0.0.1', port=12345):
    while True:
        try:
            client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            client_socket.connect((host, port))
            print("Connected to the server.")
            
            data = b""
            payload_size = struct.calcsize("L")  # 数据包的大小

            while True:
                while len(data) < payload_size:
                    packet = client_socket.recv(4096)
                    if not packet:
                        break
                    data += packet

                if not packet:
                    break

                packed_msg_size = data[:payload_size]
                data = data[payload_size:]

                msg_size = struct.unpack("L", packed_msg_size)[0]

                while len(data) < msg_size:
                    data += client_socket.recv(4096)

                frame_data = data[:msg_size]
                data = data[msg_size:]

                # 解码图像
                encoded_frame = pickle.loads(frame_data)
                frame = cv2.imdecode(encoded_frame, cv2.IMREAD_COLOR)

                cv2.imshow("Video Stream", frame)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            client_socket.close()
            print("Connection closed by server. Reconnecting...")
            time.sleep(2)  # 等待2秒后重连

        except Exception as e:
            print(f"Error: {e}. Attempting to reconnect...")
            client_socket.close()
            time.sleep(2)  # 等待2秒后重连

In [ ]:
start_client('192.168.1.11',8188)